In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/link.csv


In [2]:
movie=pd.read_csv('movie.csv')
rating=pd.read_csv('rating.csv')

In [3]:
movie.shape

(27278, 3)

In [5]:
movie_details=movie.merge(rating,on='movieId')

In [6]:
movie_details.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41


In [7]:
movie_details.drop(columns=['timestamp'],inplace=True)

In [8]:
total_ratings=movie_details.groupby(['movieId','genres']).sum()['rating'].reset_index()

In [9]:
df=movie_details.copy()

In [10]:
df.drop_duplicates(['title','genres'],inplace=True) 

In [11]:
df=df.merge(total_ratings,on='movieId')

In [12]:
df.drop(columns=['userId','rating_x','genres_y'],inplace=True)

In [13]:
df.rename(columns={'genres_x':'genres','rating_y':'rating'},inplace=True)

In [14]:
df.head()

,movieId,title,genres,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,194866.0
1,2,Jumanji (1995),Adventure|Children|Fantasy,71444.0
2,3,Grumpier Old Men (1995),Comedy|Romance,40128.5
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,7886.0
4,5,Father of the Bride Part II (1995),Comedy,37268.5


In [15]:
df['rating']=df['rating'].astype(int)

In [16]:
df.dtypes

movieId     int64
title      object
genres     object
rating      int64
dtype: object

In [17]:
df = df[df['rating']>25]

In [18]:
df.shape

(15951, 4)

In [19]:
df.to_csv('itdf.csv',index=False)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=1)
x = tfv.fit_transform(df['genres'])

In [21]:
from sklearn.metrics.pairwise import sigmoid_kernel
model = sigmoid_kernel(x, x)

In [22]:
df1=df.copy()
ti=[]
for i in df1['title']:
    ti.append(i.split(' (')[0])
df1['title']=ti

In [23]:
df1.to_csv('itdf1.csv',index=False)

In [24]:
def recommendations(title):
    i_d=[]
    indices=pd.Series(df1.index,index=df1['title']).drop_duplicates()
    idx = indices[title]
    dis_scores = list(enumerate(model[idx]))
    dis_scores = sorted(dis_scores, key=lambda x: x[1], reverse=True)
    dis_scores = dis_scores[1:31]
    idn = [i[0] for i in dis_scores]
    final =df1.iloc[idn].reset_index()
    idn = [i for i in final['index']]
    for j in idn:
        if(j<15951):
            i_d.append(j)
    indices=pd.Series(df.index,index=df['title']).drop_duplicates()
    for i in range(1,8):
        if (idn):
            print(indices.iloc[i_d].index[i])
     


In [25]:
recommendations('Toy Story')

James Dean Story, The (1957)
Knightriders (1981)
Faithless (Trolösa) (2000)
Morons From Outer Space (1985)
Trial of Joan of Arc, The (Procès de Jeanne d'Arc) (1962)
My Winnipeg (2007)
Alien Raiders (2008)


In [26]:
recommendations('Before and After')

Joe's Apartment (1996)
Rebecca (1940)
Hustler White (1996)
Preacher's Wife, The (1996)
Good Will Hunting (1997)
Broadway Melody, The (1929)
Take the Money and Run (1969)
